Christopher Zdyrski Python Assignment

Import necessary packages

In [1]:
from Bio import SeqIO
from Bio.Data import CodonTable
import pandas as pd

Function #1 Retrieve Sequences

In [2]:
def get_sequences_from_file(fasta_fn):
    sequence_data_dict = {}
    for record in SeqIO.parse(fasta_fn, "fasta"):
        description = record.description.split()
        species_name = description[1] + " " + description[2]
        sequence_data_dict[species_name] = record.seq
    return(sequence_data_dict)

Function #2 String translates a string of nucleotides to amino acids

In [3]:
def translate_function(string_nucleotides): 
     mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"] # this should work using BioPython (be sure to check what this returns)
     #for-loop through every 3rd position in string_nucleotides to get the codon using range subsets
         # IMPORTANT: if the sequence has a stop codon at the end, you should leave it off
         # this is how you can retrieve the amino acid: mito_table.forward_table[codon]
         #add the aa to aa_seq_string
     return(aa_seq_string)

Function #3 Alternative translation function 

Function #4 Calculates the molecular weight of each amino acid sequence

In [4]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
# def compute_molecular_weight(aa_seq):
#     # I think the ProtParam functions may require aa_seq to be a string.
#     # It may not work if the amino acid sequence has stop codons.
#     run the ProteinAnalysis() function on aa_seq
#	  return the molecular weight

Function #5 Computes the GC-content of each DNA sequence

Main

In [5]:
cytb_seqs = get_sequences_from_file("bears_cytb.fasta") 

bears_df = pd.read_csv("bears_mass.csv") # Includes only data for body mass 
species_list = list(bears_df.species)

FileNotFoundError: [Errno 2] No such file or directory: 'bears_cytb.fasta'

Step 6: Add columns

Step 7: For loop

Step 8: Bar-chart

Q1: What is the largest bear species?
Q2: What else is interesting about this species?

Step 9: Plot

Step 10: New Dataframe

In [ ]:
-> bears_mass_cytb.csv

Step 11: Bonus